## Model 03 : Random Forest Regression

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
house_df = pd.read_csv('Housing.csv')

for i, ele in enumerate(house_df):
    print(f'{i}: {ele}')

0: price
1: area
2: bedrooms
3: bathrooms
4: stories
5: mainroad
6: guestroom
7: basement
8: hotwaterheating
9: airconditioning
10: parking
11: prefarea
12: furnishingstatus


In [3]:
# dropping columns for hyper parmeter tuning
# house_df = house_df.drop(columns=['bathrooms','stories', 'mainroad', 'guestroom', 'basement','parking', 'prefarea'])
house_df.head(10)

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished
5,10850000,7500,3,3,1,yes,no,yes,no,yes,2,yes,semi-furnished
6,10150000,8580,4,3,4,yes,no,no,no,yes,2,yes,semi-furnished
7,10150000,16200,5,3,2,yes,no,no,no,no,0,no,unfurnished
8,9870000,8100,4,1,2,yes,yes,yes,no,yes,2,yes,furnished
9,9800000,5750,3,2,4,yes,yes,no,no,yes,1,yes,unfurnished


### Checking binary and ordinal data counts in each column

In [4]:
house_df.head(5)

house_df['hotwaterheating'].value_counts()

hotwaterheating
no     520
yes     25
Name: count, dtype: int64

In [5]:
house_df['airconditioning'].value_counts()

airconditioning
no     373
yes    172
Name: count, dtype: int64

In [6]:
house_df['furnishingstatus'].value_counts()

furnishingstatus
semi-furnished    227
unfurnished       178
furnished         140
Name: count, dtype: int64

### Encoding the binary and ordinal columns

In [7]:
# Binary data encoding 
bi_mapping = {"no" : 0, "yes" : 1}
# 
house_df['hotwaterheating_encoded'] = house_df['hotwaterheating'].map(bi_mapping)
house_df['airconditioning_encoded'] = house_df['airconditioning'].map(bi_mapping)

# house_df['mainroad_ecoded'] = house_df['mainroad'].map(bi_mapping) ##drops optimal r2_score from .6 to .585
# house_df['guestroom_ecoded'] = house_df['mainroad'].map(bi_mapping) ##drops optimal r2_score from .6 to .585
# house_df['basement_ecoded'] = house_df['basement'].map(bi_mapping) ##drops optimal r2_score from .6 to .549
house_df['prefarea_ecoded'] = house_df['prefarea'].map(bi_mapping)

In [8]:
furnishingstatus_mapping = {"unfurnished" : 0, "semi-furnished" : 1, "furnished" : 2}

# house_df['furnishingstatus_encoded'] = house_df['furnishingstatus'].map(furnishingstatus_mapping)

### Dropping columns that are encoded

In [ ]:
house_df = house_df.drop(columns=['hotwaterheating','airconditioning','furnishingstatus'])
house_df = house_df.drop(columns=['mainroad', 'guestroom', 'basement','parking', 'prefarea'])
# house_df = house_df.drop(columns=['stories'])
# house_df = house_df.drop(columns=['bedrooms'])
# house_df = house_df.drop(columns=['bathrooms'])

house_df.head(5)

,price,area,bedrooms,bathrooms,stories,hotwaterheating_encoded,airconditioning_encoded,prefarea_ecoded
0,13300000,7420,4,2,3,0,1,1
1,12250000,8960,4,4,4,0,1,0
2,12250000,9960,3,2,2,0,0,1
3,12215000,7500,4,2,2,0,1,1
4,11410000,7420,4,1,2,0,1,0


### Dividing up the dependent and independent variables to X and y


In [10]:
X = house_df.iloc[:, 1:]
y = house_df.iloc[:, 0].values
y = y.reshape(len(y),1)

### Spliting the traing and test set to a 80:20 proportion

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

In [12]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
sc_y = StandardScaler()
X_train = sc_X.fit_transform(X_train)
y_train = sc_y.fit_transform(y_train)

In [13]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 10, random_state = 0)
regressor.fit(X_train, y_train)

/home/codespace/.local/lib/python3.12/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestRegressor(n_estimators=10, random_state=0)

In [14]:
y_pred = sc_y.inverse_transform(regressor.predict(sc_X.transform(X_test)).reshape(-1,1))
np.set_printoptions(precision=2)
# print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

In [15]:
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

0.5208966382888753